In [1]:
%cd ../..

C:\Users\bram_\home\msc


# Single flow - Multi Layer Perceptron classification of CIDDS-001 OpenStack data
The implementation is based on the paper **Intelligent Cyber Attack Detetion and Classification for Network-Based Intrusion Detection Systems**. However, the training data that was used is now balanced. The data of the first and second week are joined together to obtain the biggest possible dataset to train with.

In [2]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import pandas as pd
import numpy as np
from utils import cidds_001 as utils

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.2.0


## Load datasets

In [3]:
# load and shuffle week1
week1 = pd.read_feather('saved_dfs/cidds-001/traffic/OpenStack/CIDDS-001-internal-week1-cleaned.feather')
week1 = week1.sample(frac=1, random_state=13).reset_index(drop=True)

# load and shuffle week2
week2 = pd.read_feather('saved_dfs/cidds-001/traffic/OpenStack/CIDDS-001-internal-week2-cleaned.feather')
week2 = week2.sample(frac=1, random_state=13).reset_index(drop=True)

## Preprocess datasets
* Both datasets week1 and week2 are used to obtain as much malicous traffic of the underrepresented attack types as possible.
* Split the features and labels of the new dataset and encode the labels with one hot encoding.

In [4]:
week1_benign = week1.where(week1['attack_type'] == '---').dropna()
week1_portScan = week1.where(week1['attack_type'] == 'portScan').dropna()
week1_dos = week1.where(week1['attack_type'] == 'dos').dropna()
week1_pingScan = week1.where(week1['attack_type'] == 'pingScan').dropna()
week1_bruteForce = week1.where(week1['attack_type'] == 'bruteForce').dropna()

In [5]:
print(f'len(week1_benign) = {len(week1_benign)}')
print(f'len(week1_portScan) = {len(week1_portScan)}')
print(f'len(week1_dos) = {len(week1_dos)}')
print(f'len(week1_pingScan) = {len(week1_pingScan)}')
print(f'len(week1_bruteForce) = {len(week1_bruteForce)}')

len(week1_benign) = 7010897
len(week1_portScan) = 183511
len(week1_dos) = 1252127
len(week1_pingScan) = 3359
len(week1_bruteForce) = 1626


In [6]:
week2_benign = week2.where(week2['attack_type'] == '---').dropna()
week2_portScan = week2.where(week2['attack_type'] == 'portScan').dropna()
week2_dos = week2.where(week2['attack_type'] == 'dos').dropna()
week2_pingScan = week2.where(week2['attack_type'] == 'pingScan').dropna()
week2_bruteForce = week2.where(week2['attack_type'] == 'bruteForce').dropna()

In [7]:
print(f'len(week2_benign) = {len(week2_benign)}')
print(f'len(week2_portScan) = {len(week2_portScan)}')
print(f'len(week2_dos) = {len(week2_dos)}')
print(f'len(week2_pingScan) = {len(week2_pingScan)}')
print(f'len(week2_bruteForce) = {len(week2_bruteForce)}')

len(week2_benign) = 8515329
len(week2_portScan) = 82407
len(week2_dos) = 1706900
len(week2_pingScan) = 2731
len(week2_bruteForce) = 3366


In [8]:
dataset = pd.concat(
    [
        week1_pingScan, week2_pingScan, # use all pingScan flows of both wee1 and week2
        week1_bruteForce, week2_bruteForce, # use all bruteForce flows of both week1 and week2
        week1_benign.sample(frac=1, random_state=13).head(3000), # but use a small random sample of data of all other attack types
        week2_benign.sample(frac=1, random_state=13).head(3000),
        week1_dos.sample(frac=1, random_state=13).head(3000),
        week2_dos.sample(frac=1, random_state=13).head(3000),
        week1_portScan.sample(frac=1, random_state=13).head(3000),
        week2_portScan.sample(frac=1, random_state=13).head(3000),
    ]
).sample(frac=1, random_state=13)

In [9]:
print(f'len(dataset) = {len(dataset)}')

len(dataset) = 29082


In [10]:
pd.DataFrame(dataset.groupby(by='attack_type').size(), columns=['count'])

,count
attack_type,
---,6000
bruteForce,4992
dos,6000
pingScan,6090
portScan,6000


In [11]:
data_x = dataset.drop(columns=utils.columns_to_drop + ['attack_type'])
data_y = pd.get_dummies(dataset['attack_type'])

## Noramalize features

In [12]:
_ = utils.z_score_normalization(data_x, utils.columns_to_normalize)

## Split training and testing datasets

In [13]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=13)

## Create and compile the MLP model

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(16,)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

## Fit the MLP model

In [16]:
x = x_train.to_numpy()
y = y_train.to_numpy()

In [17]:
model.fit(x=x, y=y,
          batch_size=1024,
          epochs=50)

Epoch 1/50
23/23 [==============================] - 0s 5ms/step - loss: 1.3160 - accuracy: 0.5514
Epoch 2/50
23/23 [==============================] - 0s 4ms/step - loss: 0.8697 - accuracy: 0.7642
Epoch 3/50
23/23 [==============================] - 0s 4ms/step - loss: 0.5992 - accuracy: 0.8034
Epoch 4/50
23/23 [==============================] - 0s 4ms/step - loss: 0.4805 - accuracy: 0.8244
Epoch 5/50
23/23 [==============================] - 0s 4ms/step - loss: 0.4177 - accuracy: 0.8428
Epoch 6/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3864 - accuracy: 0.8564
Epoch 7/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3641 - accuracy: 0.8633
Epoch 8/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3498 - accuracy: 0.8711
Epoch 9/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3335 - accuracy: 0.8757
Epoch 10/50
23/23 [==============================] - 0s 4ms/step - loss: 0.3244 - accuracy: 0.8824
Epoch 11/50
23/23 [

## Test the fitted model with unseen data

In [18]:
x = x_test.to_numpy()
y = y_test.to_numpy()

In [19]:
test_loss, test_acc = model.evaluate(x,  y, verbose=2)

print('\nTest accuracy:', test_acc)

182/182 - 0s - loss: 0.2037 - accuracy: 0.9347

Test accuracy: 0.9346742033958435


## Create a confusion matrix for the predictions on the test data

In [20]:
pred_y = model.predict(x)

In [21]:
y_lbl = np.argmax(y, axis=1)
pred_y_lbl = np.argmax(pred_y, axis=1)

In [22]:
tf.math.confusion_matrix(labels=y_lbl, predictions=pred_y_lbl, num_classes=5)

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[1093,   86,   16,   11,    4],
       [  13,  989,    0,    5,   11],
       [   0,    0, 1158,    0,    0],
       [  28,   32,    0, 1170,    2],
       [  12,  120,    0,   40, 1027]])>

In [23]:
print(f'Columns of data_y (and confusion matrix): {data_y.columns}')

Columns of data_y (and confusion matrix): Index(['---', 'bruteForce', 'dos', 'pingScan', 'portScan'], dtype='object')
